In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [39]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv.zip' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

# Data cleaning and normalisation

In [40]:
# we know that very low values are used to signal unavailable data
tX[tX < -900] = np.nan

In [54]:
from proj1_helpers import sample_data, standardize
seed = 1

tX_mean = np.nanmean(tX,axis=0)

tX_std = np.nanstd(tX,axis=0)
norm_tX = np.subtract(tX, tX_mean, where=np.isfinite(tX_mean))
norm_tX = np.divide(norm_tX, tX_std, where=tX_std>0)
norm_tX[np.isnan(norm_tX)] =0
#norm_tX = np.c_[np.ones(num_samples), norm_tX]

#y, norm_tX = sample_data(y, norm_tX, seed, len(y[0]))#size_samples= 100000)
norm_tX, mean_x, std_x = standardize(norm_tX)

norm_tX = np.c_[np.ones((y.shape[0], 1)), norm_tX]
initial_w = np.zeros((norm_tX.shape[1], 1))


#print(initial_w)



# Logistic Regression


In [55]:
from Implementation import logistic_regression_gd

# init parameters
max_iter = 1000
gamma = 0.00000009
initial_w = np.zeros((norm_tX.shape[1], 1))

print(np.shape(y))
print(np.shape(norm_tX))
print(np.shape(initial_w))

    
weights = logistic_regression_gd(y, norm_tX, initial_w, max_iter, gamma)



(250000,)
(250000, 31)
(31, 1)
Current iteration=0, loss=173286.79513997526, gamma=9e-08
Current iteration=100, loss=12086.40984498414, gamma=1e-09
Current iteration=200, loss=8052.144342288651, gamma=1e-09
Current iteration=300, loss=5496.583422731463, gamma=5e-10
Current iteration=400, loss=3514.0307696425443, gamma=5e-10
Current iteration=500, loss=1543.5061312556354, gamma=5e-10
Current iteration=600, loss=695.9132725131931, gamma=1e-10
Current iteration=700, loss=304.2555216173496, gamma=1e-10
Current iteration=800, loss=74.9618041679496, gamma=1e-11
Current iteration=900, loss=36.20130909176078, gamma=9e-12
loss=1.0040284043061547


# Regularized logistic regression


In [22]:
from Implementation import regu_logistic_regression_gd

# init parameters
max_iter = 1600
gamma = 0.000000005
#gamma = 0.00000009
lambda_ = 0.001


initial_w = np.zeros((norm_tX.shape[1], 1))


weights = regu_logistic_regression_gd(y, norm_tX, lambda_, initial_w, max_iter, gamma )
print(weights)

Current iteration=0, loss=173286.79513997526
Current iteration=100, loss=139532.81780052732
Current iteration=200, loss=109541.44047085172
Current iteration=300, loss=82519.39359495089
Current iteration=400, loss=57869.8509180984
Current iteration=500, loss=35145.79231381694
Current iteration=600, loss=18619.654770215184
Current iteration=700, loss=14499.590184916917
Current iteration=800, loss=10432.522224648568
Current iteration=900, loss=6676.164901606581
Current iteration=1000, loss=4685.322930473504
Current iteration=1100, loss=2706.6686377977453
Current iteration=1200, loss=928.2854446174633
Current iteration=1300, loss=536.1508126432315
Current iteration=1400, loss=144.48679264322004
Current iteration=1500, loss=59.55778142781853
loss=22.344184180736193
[[-6.18875337e-01]
 [ 7.53280508e-03]
 [-2.31162685e-01]
 [-7.36011227e-03]
 [ 9.97224563e-02]
 [ 1.09912531e-01]
 [ 1.01786746e-01]
 [-9.48998791e-02]
 [ 3.46695722e-02]
 [-2.49986449e-02]
 [ 7.06548804e-02]
 [-1.27459689e-01]
 

# Least squares and Polynomial regression


In [170]:
#weights = polynomial_regression(norm_tX)
#weights = polynomial_regression(norm_tX2)

# Split data



In [58]:
def split_data(x, y, ratio, seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # generate random indices
    num_row = len(y)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[: index_split]
    index_te = indices[index_split:]
    # create split
    x_tr = x[index_tr]
    x_te = x[index_te]
    y_tr = y[index_tr]
    y_te = y[index_te]
    return x_tr, x_te, y_tr, y_te

In [98]:
from Implementation import compute_mse, calculate_loss

def train_test_split_demo(x, y, ratio, seed):
    """polynomial regression with different split ratios and different degrees."""
    x_tr, x_te, y_tr, y_te = split_data(x, y, ratio, seed)
    # form tx
    #tx_tr = build_poly(x_tr, degree)
    #tx_te = build_poly(x_te, degree)
   
    tx_tr =x_tr
    tx_te = x_te
    #tx_tr = np.ones((len(x_tr), 1))
    #tx_tr = np.c_[tx_tr, np.power(x_tr, 1)]
    #tx_te = np.ones((len(x_te), 1))
    #tx_te = np.c_[tx_te, np.power(x_te, 1)]
    

    # init parameters
    max_iter = 1000
    gamma = 0.00000009
    initial_w = np.zeros((tx_tr.shape[1], 1))
    
    print(np.shape(y_tr))
    print(np.shape(tx_tr))
    print(np.shape(initial_w))
    


    weight = logistic_regression_gd(y_tr, tx_tr, initial_w, max_iter, gamma)
    
    y_tr = np.expand_dims(y_tr, axis=1)
    y_te = np.expand_dims(y_te, axis=1)
    
    # calculate RMSE for train and test data.
    #rmse_tr = np.sqrt(2 * compute_mse(y_tr, tx_tr, weight))
    #rmse_te = np.sqrt(2 * compute_mse(y_te, tx_te, weight))
    
    #calculate cost for train and test data
    cost_tr = calculate_loss(y_tr, tx_tr, weight)
    cost_te = calculate_loss(y_te, tx_te, weight)

    print("proportion={p}, logistic reg, Training loss={tr:.3f}, Testing loss={te:.3f}".format(
          p=ratio, tr=cost_tr, te=cost_te))
    return weight

In [102]:
seed = 6
degrees = [7]
#split_ratio = 0.9
split_ratios = [0.7, 0.8 , 0.9]
weights = train_test_split_demo(norm_tX, y, split_ratio, seed)


for split_ratio in split_ratios:
      weights = train_test_split_demo(norm_tX, y, split_ratio, seed)
        


(225000,)
(225000, 31)
(31, 1)
Current iteration=0, loss=155958.115625978, gamma=9e-08
Current iteration=100, loss=13585.566861166793, gamma=1e-09
Current iteration=200, loss=10297.21756350089, gamma=1e-09
Current iteration=300, loss=7046.212538515596, gamma=1e-09
Current iteration=400, loss=5386.16481027627, gamma=5e-10
Current iteration=500, loss=3783.517129692569, gamma=5e-10
Current iteration=600, loss=2189.648736241448, gamma=5e-10
Current iteration=700, loss=908.1271077283309, gamma=1e-10
Current iteration=800, loss=591.7859049540566, gamma=1e-10
Current iteration=900, loss=275.78545912001573, gamma=1e-10
loss=82.24524788178678
proportion=0.9, logistic reg, Training RMSE=82.245, Testing RMSE=-214.437
(175000,)
(175000, 31)
(31, 1)
Current iteration=0, loss=121300.75659798345, gamma=9e-08
Current iteration=100, loss=16688.051180456037, gamma=1e-09
Current iteration=200, loss=14626.109511646064, gamma=1e-09
Current iteration=300, loss=12583.417267372977, gamma=1e-09
Current iterati

## Generate predictions and save ouput in csv format for submission:

In [23]:
DATA_TEST_PATH = 'data/test.csv.zip' # TODO: download train data and supply path here 
y, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [24]:
# we know that very low values are used to signal unavailable data
#tX_test[tX_test < -900] = np.nan

In [25]:
tX_test_mean = np.nanmean(tX_test,axis=0)
tX_test_std = np.nanstd(tX_test,axis=0)
norm_tX_test = np.subtract(tX_test, tX_test_mean, where=np.isfinite(tX_test_mean))
norm_tX_test = np.divide(norm_tX_test, tX_test_std, where=tX_test_std>0)
norm_tX_test = np.c_[np.ones((y.shape[0], 1)), norm_tX_test]
#norm_tX_test = build_poly(norm_tX_test, 7)


In [26]:
OUTPUT_PATH = 'data/sample-submission' # TODO: fill in desired name of output file for submission
#print(np.shape(y))
#print(np.shape(norm_tX_test))
#print(np.shape(weights))
print(weights)
y_pred = predict_labels(weights, norm_tX_test)

create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

[[-6.18875337e-01]
 [ 7.53280508e-03]
 [-2.31162685e-01]
 [-7.36011227e-03]
 [ 9.97224563e-02]
 [ 1.09912531e-01]
 [ 1.01786746e-01]
 [-9.48998791e-02]
 [ 3.46695722e-02]
 [-2.49986449e-02]
 [ 7.06548804e-02]
 [-1.27459689e-01]
 [ 1.65530718e-01]
 [ 1.06251903e-01]
 [ 1.44403899e-01]
 [-8.71255966e-04]
 [-3.22939837e-03]
 [-2.30946191e-02]
 [ 5.60931037e-04]
 [ 2.89558810e-03]
 [-4.55185099e-03]
 [ 4.59451071e-03]
 [ 5.86878291e-02]
 [ 6.25180538e-02]
 [ 3.33386271e-02]
 [ 1.02786231e-04]
 [-3.99004155e-05]
 [-2.44248648e-02]
 [ 6.32800123e-04]
 [-2.07747292e-03]
 [ 5.55869534e-02]]
[[-1.6211706 ]
 [-1.14806572]
 [-1.1510502 ]
 ...
 [-0.68100117]
 [-0.30220966]
 [-1.51270127]]
